In [85]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd

In [86]:
# Establishing connection
client = MongoClient('mongodb://localhost:27017')
db = client['eduhub_db']

In [41]:
# User schema
user_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['userId', 'email', 'firstName', 'lastName', 'role', 'dateJoined', 'isActive'],
        'properties': {
            'userId': {'bsonType': 'string'},
            'email': {'bsonType': 'string'},
            'firstName': {'bsonType': 'string'},
            'lastName': {'bsonType': 'string'},
            'role': {'bsonType': 'string'},
            'dateJoined': {'bsonType': 'date'},
            'isActive': {'bsonType': 'bool'},
            'profile': {
                'bsonType': 'object',
                'properties': {
                    'bio': {'bsonType': 'string'},
                    'avatar': {'bsonType': 'string'},
                    'skills': {
                        'bsonType': 'array',
                        'items': {'bsonType': 'string'}
                    }
                }
                
            },
            'isActive': {'bsonType': 'bool'}
        }
    }
}
db.create_collection('users', validator = user_validator)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'users')

In [42]:
# Course schema
course_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['courseId', 'title', 'instructorId', 'level', 'duration', 'price', 'createdAt', 'isPublished'],
        'properties': {
            'courseId': {'bsonType': 'string'},
            'title': {'bsonType': 'string'},
            'description': {'bsonType': 'string'},
            'instructorId': {'bsonType': 'string'},  # Ideally a reference to users.userId
            'category': {'bsonType': 'string'},
            'level': {
                'bsonType': 'string',
                'enum': ['beginner', 'intermediate', 'advanced']
            },
            'duration': {'bsonType': 'double'},  # You can use 'int' if you prefer integers
            'price': {'bsonType': 'double'},
            'tags': {
                'bsonType': 'array',
                'items': {'bsonType': 'string'}
            },
            'createdAt': {'bsonType': 'date'},
            'updatedAt': {'bsonType': 'date'},
            'isPublished': {'bsonType': 'bool'}
        }
    }
}
db.create_collection("courses", validator=course_validator)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'courses')

In [48]:
# Enrollment schema
enrollment_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["enrollmentId", "studentId", "courseId", "enrolledAt"],
        "properties": {
            "enrollmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "enrolledAt": {"bsonType": "date"},
            "progress": {"bsonType": "int"},
            "status": {"bsonType": "string", "enum": ["ongoing", "completed", "dropped"]}
        }
    }
}

db.create_collection("enrollments", validator=enrollment_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'enrollments')

In [49]:
# Lesson Schema
lesson_schema = {
    "$jsonSchema": {
      "bsonType": "object",
      "required": ["lessonId", "courseId", "title", "createdAt"],
      "properties": {
        "lessonId": { "bsonType": "string", "description": "Must be unique" },
        "courseId": { "bsonType": "string", "description": "Reference to courses" },
        "title": { "bsonType": "string", "description": "Required" },
        "content": { "bsonType": "string", "description": "Lesson material" },
        "duration": { "bsonType": "int", "description": "In minutes" },
        "createdAt": { "bsonType": "date" },
        "updatedAt": { "bsonType": "date" }
      }
    }
  }
db.create_collection("lessons", validator=lesson_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'lessons')

In [73]:
# Assignment schema
assignment_schema = {
    "$jsonSchema": {
      "bsonType": "object",
      "required": ["assignmentId", "courseId", "title", "dueDate"],
      "properties": {
        "assignmentId": { "bsonType": "string", "description": "Must be unique" },
        "courseId": { "bsonType": "string", "description": "Reference to courses" },
        "title": { "bsonType": "string", "description": "Required" },
        "description": { "bsonType": "string" },
        "dueDate": { "bsonType": "string" },
        "maxScore": { "bsonType": "int" },
        "createdAt": { "bsonType": "date" },
        "updatedAt": { "bsonType": "date" }
      }
    }
}
db.create_collection("assignments", validator=assignment_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'assignments')

In [54]:
# Assignment submission schema
assignment_submission_schema = {
    "$jsonSchema": {
      "bsonType": "object",
      "required": ["submissionId", "assignmentId", "studentId", "submittedAt"],
      "properties": {
        "submissionId": { "bsonType": "string", "description": "Must be unique" },
        "assignmentId": { "bsonType": "string", "description": "Reference to assignments" },
        "studentId": { "bsonType": "string", "description": "Reference to users" },
        "submittedAt": { "bsonType": "date" },
        "content": { "bsonType": "string", "description": "Text or file link" },
        "score": { "bsonType": "int" },
        "feedback": { "bsonType": "string" }
      }
    }
  }
db.create_collection("assignment_submission", validator=assignment_submission_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'assignment_submission')

In [56]:
# Inserting data for the user schema
user_data = [
  {
    "userId": "U001",
    "email": "micheal_jordan@gmail.com",
    "firstName": "Micheal",
    "lastName": "Jordan",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Aspiring data scientist.",
      "avatar": "avatar1.png",
      "skills": ["Python", "Statistics"]
    },
    "isActive": True
  },
  {
    "userId": "U002",
    "email": "john_doe@gmail.com",
    "firstName": "John",
    "lastName": "Doe",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Experienced software developer.",
      "avatar": "avatar2.png",
      "skills": ["JavaScript", "MongoDB"]
    },
    "isActive": True
  },
  {
    "userId": "U003",
    "email": "emma_jones@gmail.com",
    "firstName": "Emma",
    "lastName": "Jones",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Machine learning enthusiast.",
      "avatar": "avatar3.png",
      "skills": ["TensorFlow", "Data Science"]
    },
    "isActive": True
  },
  {
    "userId": "U004",
    "email": "michael_brown@gmail.com",
    "firstName": "Michael",
    "lastName": "Brown",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Cybersecurity specialist.",
      "avatar": "avatar4.png",
      "skills": ["Ethical Hacking", "Network Security"]
    },
    "isActive": False
  },
  {
    "userId": "U005",
    "email": "sophia_wilson@gmail.com",
    "firstName": "Sophia",
    "lastName": "Wilson",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Passionate about web design.",
      "avatar": "avatar5.png",
      "skills": ["HTML", "CSS", "UI/UX"]
    },
    "isActive": True
  },
  {
    "userId": "U006",
    "email": "james_taylor@gmail.com",
    "firstName": "James",
    "lastName": "Taylor",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Cloud computing expert.",
      "avatar": "avatar6.png",
      "skills": ["AWS", "Docker"]
    },
    "isActive": True
  },
  {
    "userId": "U007",
    "email": "olivia_martin@gmail.com",
    "firstName": "Olivia",
    "lastName": "Martin",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "AI and NLP enthusiast.",
      "avatar": "avatar7.png",
      "skills": ["NLP", "Deep Learning"]
    },
    "isActive": True
  },
  {
    "userId": "U008",
    "email": "william_thomas@gmail.com",
    "firstName": "William",
    "lastName": "Thomas",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Specialist in mobile development.",
      "avatar": "avatar8.png",
      "skills": ["Flutter", "Swift"]
    },
    "isActive": False
  },
  {
    "userId": "U009",
    "email": "ava_white@gmail.com",
    "firstName": "Ava",
    "lastName": "White",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Interested in game development.",
      "avatar": "avatar9.png",
      "skills": ["Unity", "C#"]
    },
    "isActive": True
  },
  {
    "userId": "U010",
    "email": "logan_harris@gmail.com",
    "firstName": "Logan",
    "lastName": "Harris",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Blockchain technology educator.",
      "avatar": "avatar10.png",
      "skills": ["Solidity", "Cryptography"]
    },
    "isActive": True
  },
    {
    "userId": "U011",
    "email": "micheal_jordan@gmail.com",
    "firstName": "Micheal",
    "lastName": "Jordan",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Aspiring data scientist.",
      "avatar": "avatar1.png",
      "skills": ["Python", "Statistics"]
    },
    "isActive": True
  },
  {
    "userId": "U012",
    "email": "john_doe@gmail.com",
    "firstName": "John",
    "lastName": "Doe",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Experienced software developer.",
      "avatar": "avatar2.png",
      "skills": ["JavaScript", "MongoDB"]
    },
    "isActive": True
  },
  {
    "userId": "U013",
    "email": "emma_jones@gmail.com",
    "firstName": "Emma",
    "lastName": "Jones",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Machine learning enthusiast.",
      "avatar": "avatar3.png",
      "skills": ["TensorFlow", "Data Science"]
    },
    "isActive": True
  },
  {
    "userId": "U014",
    "email": "michael_brown@gmail.com",
    "firstName": "Michael",
    "lastName": "Brown",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Cybersecurity specialist.",
      "avatar": "avatar4.png",
      "skills": ["Ethical Hacking", "Network Security"]
    },
    "isActive": False
  },
  {
    "userId": "U015",
    "email": "sophia_wilson@gmail.com",
    "firstName": "Sophia",
    "lastName": "Wilson",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Passionate about web design.",
      "avatar": "avatar5.png",
      "skills": ["HTML", "CSS", "UI/UX"]
    },
    "isActive": True
  },
  {
    "userId": "U016",
    "email": "james_taylor@gmail.com",
    "firstName": "James",
    "lastName": "Taylor",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Cloud computing expert.",
      "avatar": "avatar6.png",
      "skills": ["AWS", "Docker"]
    },
    "isActive": True
  },
  {
    "userId": "U017",
    "email": "olivia_martin@gmail.com",
    "firstName": "Olivia",
    "lastName": "Martin",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "AI and NLP enthusiast.",
      "avatar": "avatar7.png",
      "skills": ["NLP", "Deep Learning"]
    },
    "isActive": True
  },
  {
    "userId": "U018",
    "email": "william_thomas@gmail.com",
    "firstName": "William",
    "lastName": "Thomas",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Specialist in mobile development.",
      "avatar": "avatar8.png",
      "skills": ["Flutter", "Swift"]
    },
    "isActive": False
  },
  {
    "userId": "U019",
    "email": "ava_white@gmail.com",
    "firstName": "Ava",
    "lastName": "White",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Interested in game development.",
      "avatar": "avatar9.png",
      "skills": ["Unity", "C#"]
    },
    "isActive": True
  },
  {
    "userId": "U020",
    "email": "logan_harris@gmail.com",
    "firstName": "Logan",
    "lastName": "Harris",
    "role": "instructor",
    "dateJoined": datetime.now(),
    "profile": {
      "bio": "Blockchain technology educator.",
      "avatar": "avatar10.png",
      "skills": ["Solidity", "Cryptography"]
    },
    "isActive": True
  }
]
db.users.insert_many(user_data)
('Users inserted successfully')

'Users inserted successfully'

In [58]:
# Inserting data for courses schema
courses_data = [
    {
        "courseId": "C001",
        "title": "Python for Data Science",
        "description": "Learn Python for data analysis, visualization, and machine learning.",
        "instructorId": "U001",
        "category": "Programming",
        "level": "beginner",
        "duration": 8.0,  # weeks
        "price": 99.99,
        "tags": ["Python", "Data Science", "Machine Learning"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    },
    {
        "courseId": "C002",
        "title": "Web Development with JavaScript",
        "description": "Master JavaScript, HTML, CSS, and Node.js to build full-stack applications.",
        "instructorId": "U002",
        "category": "Web Development",
        "level": "intermediate",
        "duration": 12.0,
        "price": 149.99,
        "tags": ["JavaScript", "HTML", "CSS", "Node.js"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    },
    {
        "courseId": "C003",
        "title": "Machine Learning with TensorFlow",
        "description": "Explore deep learning techniques and build AI models with TensorFlow.",
        "instructorId": "U003",
        "category": "Artificial Intelligence",
        "level": "advanced",
        "duration": 10.0,
        "price": 199.99,
        "tags": ["Machine Learning", "TensorFlow", "Deep Learning"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    },
    {
        "courseId": "C004",
        "title": "Cybersecurity and Ethical Hacking",
        "description": "Understand cybersecurity principles and ethical hacking techniques.",
        "instructorId": "U004",
        "category": "Cybersecurity",
        "level": "advanced",
        "duration": 14.0,
        "price": 249.99,
        "tags": ["Cybersecurity", "Ethical Hacking", "Network Security"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": False
    },
    {
        "courseId": "C005",
        "title": "Cloud Computing with AWS",
        "description": "Learn cloud services, deployment strategies, and AWS security.",
        "instructorId": "U005",
        "category": "Cloud Computing",
        "level": "intermediate",
        "duration": 16.0,
        "price": 179.99,
        "tags": ["AWS", "Cloud Computing", "Docker"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    },
    {
        "courseId": "C006",
        "title": "UI/UX Design Fundamentals",
        "description": "Gain expertise in user interface and experience design techniques.",
        "instructorId": "U006",
        "category": "Design",
        "level": "beginner",
        "duration": 6.0,
        "price": 89.99,
        "tags": ["UI/UX", "Design", "CSS"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    },
    {
        "courseId": "C007",
        "title": "Mobile App Development with Flutter",
        "description": "Develop high-quality mobile applications with Flutter and Dart.",
        "instructorId": "U007",
        "category": "Mobile Development",
        "level": "intermediate",
        "duration": 12.0,
        "price": 159.99,
        "tags": ["Flutter", "Dart", "Mobile Development"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": False
    },
    {
        "courseId": "C008",
        "title": "Blockchain & Cryptocurrency Essentials",
        "description": "Understand blockchain technology, smart contracts, and cryptocurrencies.",
        "instructorId": "U008",
        "category": "Blockchain",
        "level": "advanced",
        "duration": 10.0,
        "price": 199.99,
        "tags": ["Blockchain", "Cryptocurrency", "Solidity"],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    }
]

db.courses.insert_many(courses_data)
("Courses inserted successfully.")

'Courses inserted successfully.'

In [74]:
import uuid

In [67]:
# inserting data for enrollemnts data
enrollment_data = [
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U001",
        "courseId": "C001",
        "enrolledAt": datetime.now(),
        "progress": 100,
        "status": "completed"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U002",
        "courseId": "C002",
        "enrolledAt": datetime.now(),
        "progress": 50,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U003",
        "courseId": "C003",
        "enrolledAt": datetime.now(),
        "progress": 20,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U004",
        "courseId": "C004",
        "enrolledAt": datetime.now(),
        "progress": 80,
        "status": "dropped"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U005",
        "courseId": "C005",
        "enrolledAt": datetime.now(),
        "progress": 10,
        "status": "dropped"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U006",
        "courseId": "C006",
        "enrolledAt": datetime.now(),
        "progress": 65,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U007",
        "courseId": "C007",
        "enrolledAt": datetime.now(),
        "progress": 100,
        "status": "completed"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U008",
        "courseId": "C008",
        "enrolledAt": datetime.now(),
        "progress": 90,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U009",
        "courseId": "C001",
        "enrolledAt": datetime.now(),
        "progress": 70,
        "status": "dropped"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U010",
        "courseId": "C002",
        "enrolledAt": datetime.now(),
        "progress": 90,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U001",
        "courseId": "C003",
        "enrolledAt": datetime.now(),
        "progress": 25,
        "status": "dropped"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U002",
        "courseId": "C004",
        "enrolledAt": datetime.now(),
        "progress": 100,
        "status": "completed"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U003",
        "courseId": "C005",
        "enrolledAt": datetime.now(),
        "progress": 55,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U004",
        "courseId": "C006",
        "enrolledAt": datetime.now(),
        "progress": 45,
        "status": "ongoing"
    },
    {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": "U005",
        "courseId": "C007",
        "enrolledAt": datetime.now(),
        "progress": 100,
        "status": "completed"
    }
]
db.enrollments.insert_many(enrollment_data)
("Enrollments inserted successfully.")

'Enrollments inserted successfully.'

In [69]:
# Inserting data for lessons data
lessons_data = [
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Introduction to Python",
        "content": "An overview of Python programming language and its applications.",
        "duration": 45,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Variables and Data Types",
        "content": "Understanding different data types and variables in Python.",
        "duration": 60,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "HTML Basics",
        "content": "Introduction to HTML structure and elements.",
        "duration": 50,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "CSS Styling",
        "content": "Learning CSS properties and how to style web pages.",
        "duration": 55,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Introduction to Machine Learning",
        "content": "Overview of machine learning concepts and algorithms.",
        "duration": 70,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Data Preprocessing",
        "content": "Techniques for handling missing data and feature engineering.",
        "duration": 90,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Cybersecurity Fundamentals",
        "content": "Understanding cybersecurity principles and best practices.",
        "duration": 65,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Ethical Hacking Techniques",
        "content": "Learning penetration testing methodologies.",
        "duration": 80,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Cloud Computing Concepts",
        "content": "Introduction to cloud computing and deployment strategies.",
        "duration": 75,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "AWS Cloud Services",
        "content": "Overview of AWS services including EC2 and S3.",
        "duration": 85,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "UI/UX Design Principles",
        "content": "Understanding user experience and interface design techniques.",
        "duration": 70,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Wireframing and Prototyping",
        "content": "Creating wireframes and interactive prototypes.",
        "duration": 60,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Flutter Basics",
        "content": "Introduction to Flutter and Dart programming language.",
        "duration": 90,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Building Responsive UI",
        "content": "Techniques for creating responsive mobile applications.",
        "duration": 85,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Introduction to Blockchain",
        "content": "Understanding blockchain technology and its applications.",
        "duration": 75,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Smart Contracts with Solidity",
        "content": "Learn Solidity for smart contract development.",
        "duration": 95,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
    
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Introduction to Python",
        "content": "An overview of Python programming language and its applications.",
        "duration": 45,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Variables and Data Types",
        "content": "Understanding different data types and variables in Python.",
        "duration": 60,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "HTML Basics",
        "content": "Introduction to HTML structure and elements.",
        "duration": 50,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "CSS Styling",
        "content": "Learning CSS properties and how to style web pages.",
        "duration": 55,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Introduction to Machine Learning",
        "content": "Overview of machine learning concepts and algorithms.",
        "duration": 70,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Data Preprocessing",
        "content": "Techniques for handling missing data and feature engineering.",
        "duration": 90,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Cybersecurity Fundamentals",
        "content": "Understanding cybersecurity principles and best practices.",
        "duration": 65,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Ethical Hacking Techniques",
        "content": "Learning penetration testing methodologies.",
        "duration": 80,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "lessonId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Cloud Computing Concepts",
        "content": "Introduction to cloud computing and deployment strategies.",
        "duration": 75,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }
]
db.lessons.insert_many(lessons_data)
("lessons inserted successfully.")

'lessons inserted successfully.'

In [75]:
# inserting data for assignment schema
assignment_data = [
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 1",
        "description": "Introduction to Data Structures",
        "dueDate": "2025-07-10T13:45:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 2",
        "description": "Database Normalization",
        "dueDate": "2025-07-11T14:00:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 3",
        "description": "Advanced Algorithm Analysis",
        "dueDate": "2025-07-12T15:30:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 4",
        "description": "Artificial Intelligence Fundamentals",
        "dueDate": "2025-07-13T10:00:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 5",
        "description": "Cybersecurity Principles",
        "dueDate": "2025-07-14T11:00:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 6",
        "description": "Web Development Basics",
        "dueDate": "2025-07-15T16:00:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 7",
        "description": "Machine Learning Applications",
        "dueDate": "2025-07-16T12:30:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 8",
        "description": "Cloud Computing Concepts",
        "dueDate": "2025-06-17T17:30:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 9",
        "description": "Mobile App Development",
        "dueDate": "2025-07-18T09:30:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    },
    {
        "assignmentId": str(uuid.uuid4()),
        "courseId": str(uuid.uuid4()),
        "title": "Assignment 10",
        "description": "Software Engineering Practices",
        "dueDate": "2025-07-19T14:45:00",
        "maxScore": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }
]
db.assignments.insert_many(assignment_data)
("assignment inserted successfully.")

'assignment inserted successfully.'

In [76]:
# Inserting data for assignment submission schema
data_submit_data = [
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt":datetime.now(),
        "content": "Solution for Assignment 1",
        "score": 95,
        "feedback": "Excellent work!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Report on Database Normalization",
        "score": 88,
        "feedback": "Good effort, minor improvements needed."
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Algorithm design analysis",
        "score": 90,
        "feedback": "Solid explanation of concepts!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "AI Fundamentals essay",
        "score": 92,
        "feedback": "Very insightful discussion!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Cybersecurity case study",
        "score": 85,
        "feedback": "Well-structured analysis."
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Web development tutorial",
        "score": 98,
        "feedback": "Very well explained!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Machine learning application report",
        "score": 90,
        "feedback": "Demonstrates great understanding!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Cloud computing overview",
        "score": 88,
        "feedback": "Good grasp of core concepts."
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Mobile app design principles",
        "score": 93,
        "feedback": "Great design approach!"
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Software engineering case study",
        "score": 87,
        "feedback": "Solid case study approach."
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Network security research",
        "score": 95,
        "feedback": "Thorough research and analysis."
    },
    {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": str(uuid.uuid4()),
        "studentId": str(uuid.uuid4()),
        "submittedAt": datetime.now(),
        "content": "Data science final project",
        "score": 96,
        "feedback": "Exceptional work!"
    }
]
db.assignment_submission.insert_many(data_submit_data)
("assignment submission inserted successfully.")

'assignment submission inserted successfully.'

In [99]:
# Adding a new student user
users_collection = db['users']
new_user = {
    'userId': 'u001',
    'email': 'mbamahenrry@gmail.com',
    'firstName': 'Mbama',
    'lastName': 'Herry',
    'role': 'student',
    'dateJoined': datetime.utcnow(),
    'isActive': True,
    'profile': {
        'bio': 'Passionate learner of Python and MongoDB.',
        'avatar': 'https://example.com/avatar.jpg',
        'skills': ['Python', 'MongoDB', 'Data Analysis']
    }
}
insert_result = users_collection.insert_one(new_user)
if insert_result.acknowledged:
    print(f"User has been added with _id: {insert_result.inserted_id}")
else:
    print("Failed to add user.")

User has been added with _id: 68496e8f70de6a3fa6ca93a3


In [103]:
# Creating a new course
course_collection =db['courses']
new_course =  {
        "courseId": "C071",
        "title": "Data Engineering with Mongodb",
        "description": "Learn how to design NoSql databases and build piplines using Mongodb",
        "instructorId": "U071",
        "category": "Data Engineering",
        "level": "beginner",
        "duration": 15.5,  # weeks
        "price": 79.99,
        "tags": ['Mongodb', 'data engineering', 'nosql', 'etl'],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    }
insert_result = course_collection.insert_one(new_course)
if insert_result.acknowledged:
    print(f"User has been added with _id: {insert_result.inserted_id}")
else:
    print("Failed to add user.")

User has been added with _id: 6849717970de6a3fa6ca93a4


In [ ]:
# Enrolling a student in a course
